Name: Kendall Haworth

Bronco ID: 011694826

Partner Name: Clara Nguyen

Bronco ID: 011963965

In [1]:
import json
from typing import NamedTuple, Dict, Tuple, Optional, Sequence, List, Set, FrozenSet
import array
import heapq
import time
import itertools

with open('Crafting.json') as f:
    Crafting = json.load(f)

# List of items that can be in your inventory:
print(Crafting['Items'])
# example: ['bench', 'cart', ..., 'wood', 'wooden_axe', 'wooden_pickaxe']

# List of items needed to be in your inventory at the end of the plan:
# (okay to have more than this; some might be satisfied by initial inventory)
print(Crafting['Goal'])
# {'stone_pickaxe': 2}

# Dict of crafting recipes (each is a dict):
print(Crafting['Recipes']['craft stone_pickaxe at bench'])
# example:
# {	'Produces': {'stone_pickaxe': 1},
#	'Requires': {'bench': True},
#	'Consumes': {'cobble': 3, 'stick': 2},
#	'Time': 1
# }

['bench', 'cart', 'coal', 'cobble', 'furnace', 'ingot', 'iron_axe', 'iron_pickaxe', 'ore', 'plank', 'rail', 'stick', 'stone_axe', 'stone_pickaxe', 'wood', 'wooden_axe', 'wooden_pickaxe']
{'stone_pickaxe': 1}
{'Produces': {'stone_pickaxe': 1}, 'Requires': {'bench': True}, 'Consumes': {'cobble': 3, 'stick': 2}, 'Time': 1}


In [2]:
items_by_index = list(sorted(Crafting['Items']))
items_to_indices = {item: index for index, item in enumerate(items_by_index)}

In [3]:
class State:
    items: array.array

    def __init__(self, items: Optional[Sequence[int]] = None) -> None:
        if items is not None:
            # Copying a state from an old state.
            # This call to the array constructor creates an array of unsigned integers and initializes it 
            # from the contents of items.
            self.items = array.array('I', items)
        else:
            self.items = array.array('I', [0 for item in items_by_index])

    def __add__(self, other:'State')->'State':
        s = State(self.items)
        
        # TODO How do you add two states together
        for i in range(len(items_by_index)):
            s.items[i] += other.items[i]
        return s
    
    def __sub__(self, other:'State')->'State':
        s = State(self.items)
        
        # TODO How do you subtract one state from another
        for i in range(len(items_by_index)):
            s.items[i] -= other.items[i]
        return s

    def __ge__(self, other:'State')->bool:
        
        # TODO How do we know whether one state (self) contains everything that's inside of another (other)?
        for i in range(len(items_by_index)):
            if (self.items[i] < other.items[i]):
                return False
        
        return True
        
    def __lt__(self, other:'State')->bool:
        return not (self >= other)

    def __eq__(self, other:'State')->bool:
        return self.items == other.items

    def __hash__(self)->int:
        hsh = 5381
        for s in self.items:
            hsh = ((hsh << 5) + hsh) + s
        return hsh

    def __str__(self)-> str:
        out_str = []
        for k,v  in self.to_dict().items():
            out_str.append('{}:{}'.format(k,v))
        return ', '.join(out_str)

    def to_dict(self) -> Dict[str, int]:
        return {items_by_index[idx]: self.items[idx]
                for idx in range(len(self.items))}

    @classmethod
    def from_dict(cls, item_dict: Dict[str, int]) -> 'State':
        return cls([
            item_dict.get(item, 0) for item in items_by_index
        ])

In [4]:
# Example
initial = State.from_dict({'stone_pickaxe':1, 'ingot':2})
goal = State.from_dict({'ingot':1})
print(initial)
print()
print(goal)
print()
print(initial >= goal)
print()
print(goal < initial)
print()
print(goal >= initial)
print()
print(initial + goal)
print()
print(initial - goal)

bench:0, cart:0, coal:0, cobble:0, furnace:0, ingot:2, iron_axe:0, iron_pickaxe:0, ore:0, plank:0, rail:0, stick:0, stone_axe:0, stone_pickaxe:1, wood:0, wooden_axe:0, wooden_pickaxe:0

bench:0, cart:0, coal:0, cobble:0, furnace:0, ingot:1, iron_axe:0, iron_pickaxe:0, ore:0, plank:0, rail:0, stick:0, stone_axe:0, stone_pickaxe:0, wood:0, wooden_axe:0, wooden_pickaxe:0

True

True

False

bench:0, cart:0, coal:0, cobble:0, furnace:0, ingot:3, iron_axe:0, iron_pickaxe:0, ore:0, plank:0, rail:0, stick:0, stone_axe:0, stone_pickaxe:1, wood:0, wooden_axe:0, wooden_pickaxe:0

bench:0, cart:0, coal:0, cobble:0, furnace:0, ingot:1, iron_axe:0, iron_pickaxe:0, ore:0, plank:0, rail:0, stick:0, stone_axe:0, stone_pickaxe:1, wood:0, wooden_axe:0, wooden_pickaxe:0


In [5]:
#call like this: Recipe(State, State, State, int) to create a NamedTuple of this type

class Recipe(NamedTuple):
    produces: State
    consumes: State
    requires: State
    cost: int

In [6]:
#makes a dictionary mapping the name of each item along with their recipes (i.e. what it produces, consumes, requires, and costs)

recipes: Dict[str, Recipe] = {}
for name, rule in Crafting['Recipes'].items():
    recipes[name] = Recipe(
        State.from_dict(rule.get('Produces', {})),
        State.from_dict(rule.get('Consumes', {})),
        State.from_dict({item: 1 if req else 0
                         for item, req in rule.get('Requires', {}).items()}),
        rule['Time']
    )

In [7]:
def preconditions_satisfied(state: State, recipe: Recipe) -> bool:
    # TODO What needs to be true about state and recipe?
    # Feel free to use State's >= method
    return state >= recipe.consumes and state >= recipe.requires

def apply_effects(state: State, recipe: Recipe) -> State:
    #TODO What happens when a recipe is used?
    state -= recipe.consumes
    state += recipe.produces
    return state


initial = State.from_dict({'stone_pickaxe':1, 'ingot':2})

#Let's try using the pickaxe
if preconditions_satisfied(initial,recipes['stone_pickaxe for cobble']):
    print('Stone Pickaxe',apply_effects(initial, recipes['stone_pickaxe for cobble']))

if preconditions_satisfied(initial,recipes['wooden_pickaxe for cobble']):
    print('Wooden Pickaxe',apply_effects(initial, recipes['wooden_pickaxe for cobble']))

Stone Pickaxe bench:0, cart:0, coal:0, cobble:1, furnace:0, ingot:2, iron_axe:0, iron_pickaxe:0, ore:0, plank:0, rail:0, stick:0, stone_axe:0, stone_pickaxe:1, wood:0, wooden_axe:0, wooden_pickaxe:0


In [8]:
import queue

def plan_bfs(initial: State, goal: State, limit:int) -> Tuple[int, Optional[List[str]]]:
    
    # E. Implement it here!
    #When you find a solution, print out the number of nodes visited.
    #If you don't find a solution, print out the number of nodes visited.
    #Return a tuple of (nodes_visited, None) if no path exists,
    #or else a tuple of (nodes_visited, cost, path) where path is a list of recipe names.
    # You should also use limit to avoid visiting too many nodes before returning _something_.
    
    #check all states. if the preconditions are satisfied, add it to the queue along with the previous state
    #Test all states until the queue is empty, the goal has been reached, or the iterations have reached max
    
    path = []
    cost = 0
    
    myqueue = queue.Queue()
    myqueue.put((initial, cost, path)) #add the initial state to the queue with the path taken
    nodes_visited = 0
    
    while (nodes_visited < limit and not myqueue.empty()):
        
        #get the current state
        currentState, cost, path = myqueue.get()
        nodes_visited += 1
        
        #check if any recipes are satisfied by the current state.
        #If they are, add the resulting state to the queue (after consuming the recipe on the current state)
        
        for key in recipes:
            if (preconditions_satisfied(currentState, recipes[key])):
                
                thisCost = cost
                thisPath = path.copy()
                
                #update the path taken and the cost to that path
                thisPath.append(key)
                thisCost += recipes[key].cost
                
                #consume the state
                newState = apply_effects(currentState, recipes[key])
                
                #if the new state satisfies the goal, we are done!
                if (newState >= goal):
                    return (nodes_visited, thisCost, thisPath)
                
                #add it to the queue
                myqueue.put((newState, thisCost, thisPath))

    return (limit, None)

In [9]:
#Let's try it out. Solution does not have to be optimal!

print(plan_bfs(State.from_dict({}),
                    State.from_dict({'stone_pickaxe':1}),
                    200000))
print(plan_bfs(State.from_dict({'bench':1,'stone_pickaxe':1}),
                    State.from_dict({'ingot':1}),
                    200000))

(68732, 31, ['punch for wood', 'craft plank', 'craft stick', 'punch for wood', 'craft plank', 'punch for wood', 'craft plank', 'craft bench', 'craft wooden_pickaxe at bench', 'wooden_pickaxe for cobble', 'wooden_pickaxe for cobble', 'wooden_pickaxe for cobble', 'craft stone_pickaxe at bench'])
(200000, None)


In [10]:
class Proposition(NamedTuple):
    item: int
    at_least: int

def state_propositions(state: State) -> Set[Proposition]:
    propositions: Set[Proposition] = set()
    # TODO Do something for each item in state.  Output all propositions entailed by the state's contents
    
    for i in range(len(items_by_index)):
        if (state.items[i] > 0):
            for j in range(state.items[i]):
                propositions.add(Proposition(i, j+1))
    
    return propositions

In [11]:
def recipe_to_propositions(recipe: Recipe) -> Set[Proposition]:
    propositions: Set[Proposition] = set()
    # TODO Do something with recipe.consumes, recipe.produces, and recipe.requires.
    # Emit, for this recipe, all the propositions entailed by the postconditions 
    #and the _minimal_ set of propositions required for the preconditions 
    #(i.e., don't need to output wood >= 2, wood >= 1, wood >= 0 if the recipe needs 2 wood.)
    
    for i in range(len(items_by_index)):
        if (recipe.consumes.items[i] > 0):
            for j in range(recipe.consumes.items[i]):
                 propositions.add(Proposition(i, j+1))
                    
        if (recipe.requires.items[i] > 0):
            for j in range(recipe.requires.items[i]):
                propositions.add(Proposition(i, j+1))
                    
        if (recipe.produces.items[i] > 0): #can only produce one of an item
            propositions.add(Proposition(i, recipe.produces.items[i]))
    
    return propositions

recipe_propositions = set()
for r in recipes:
    recipe_propositions |= recipe_to_propositions(recipes[r])

In [12]:
#updates seen_combinations, updates the combinations list if the combination is unique and is not a subset of all other combinations
#returns true if at least one unique (not in seen_combinations) combo from all ALLCOMBINATIONS is found in your state
#else, return false
#also, modifies seen_combinations such that any combo found in the state is added to seen_combinations

def see_state(state:State, combinations:List[Set[Proposition]], seen_combinations:Set[FrozenSet[Proposition]]) -> bool:
    any_new = False

    #TODO Is this combination already in seen_combinations?
    #If not, it's novel; so is this combination a subset of the state_props?
    # Example, assuming propositions is a Set[Proposition]
    
    state_props:Set[Proposition] = state_propositions(state)
    
    for combo in combinations:
        if combo not in seen_combinations:
            if state_props.issuperset(combo):
                any_new = True
                seen_combinations.add(combo)

    return any_new

In [13]:
#priority queue from homework 1, used for A* extra credit and sorting recipes from lowest to highest cost

import heapq

class PriorityQueue:
    def __init__(self):
        self.elements = []
        self.visited = 0
    
    #returns true if the queue is empty, false if it is not
    def empty(self):
        return len(self.elements) == 0
    
    #adds an item to the priorityQueue
    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, self.visited, item))
        self.visited += 1
        
    #gets the first element from the priorityQueue
    def get(self):
        item = heapq.heappop(self.elements)
        return item[0], item[2]

In [14]:
#this cell organizes the recipes in the default order, a random order, from low to high cost, and from high to low cost

import random

#recipes: the default order
recipes_random: Dict[str, Recipe] = {} #a random order
        
#index the recipe keys to randomize them    
recipes_by_number = []
    
for key in recipes: 
    recipes_by_number.append(key)
    
#keep a list of the seen values, so values don't repeat
seen_numbers = []

#now randomize the recipes
for key in recipes:
    num = random.randint(0, len(recipes)-1) #generates a number from 0-the length of all the recipes-1, inclusive
    
    while num in seen_numbers: #make sure the number hasn't been seen before
        num += 1
        
        if (num > len(recipes)-1): #if exceeded the range, set it back to 0
            num = 0
            
    seen_numbers.append(num)
    recipes_random[recipes_by_number[num]] = recipes[recipes_by_number[num]]

#get the cost of the recipes from low to high and high to low
#We used the priority queue from hw 1 to sort the recipes from low to high for us,
#since we needed the queue for the A* extra credit anyways

recipes_low_to_high: Dict[str, Recipe] = {} #organized from low to high cost
recipes_high_to_low: Dict[str, Recipe] = {} #organized from high to low cost
    
priorityQueue = PriorityQueue()

for key in recipes:
    priorityQueue.put(key, recipes[key].cost)
    
while (not priorityQueue.empty()):
    key = priorityQueue.get()[1]
    recipes_low_to_high[key] = recipes[key]

#make a list of the keys in order of low to high cost
keyList = []
for key in recipes_low_to_high:
    keyList.append(key)

#pop the keys off the get the dictionary in reverse order
while keyList:
    key = keyList.pop()
    recipes_high_to_low[key] = recipes[key]

In [15]:
#a breadth-first search version of IW, uses a list like a queue

import random

def width_search_BFS(initial: State, goal: State, WMax: int) -> Tuple[int, Optional[List[str]]]:
    start_time = time.time()
    all_propositions = recipe_propositions | state_propositions(initial) | state_propositions(goal)
    all_combinations: List[FrozenSet[Proposition]] = []
        
    visited = 0
    # Increase W up to WMax
    for W in range(1, WMax + 1):
        
        # Calculate all combinations of propositions at size W and add to all_combination
        all_combinations += [frozenset(props) for props in itertools.combinations(all_propositions, W)]
        print("W=",W,"Combination count=",len(all_combinations))
        
        # Track, for each combination (by index), whether we have seen this combination before (0 for no, >0 for yes)
        seen_combinations: Set[FrozenSet[Proposition]] = set()
            
        # Initialize seen_combinations
        #see_state(initial, all_combinations, seen_combinations)
        
        open_list: List[Tuple[int, State]] = [(0, initial)]
        best_costs: Dict[State, int] = {initial: 0}
        best_from: Dict[State, List[str]] = {initial: []}
            
        #TODO This should look like your graph search, except...
        #Call see_state on newly expanded states to update seen_combinations
        #and use its return value to decide whether to add this state to the 
        #open list (is that the only thing that determines whether it should go on the open list?)
            
        while open_list:
        
            #get the current state
            currentState = open_list.pop(0)[1] #always get the first element added in the list, like a queue
            
            cost = best_costs[currentState]
            path = best_from[currentState]
            visited += 1
        
            #check if any recipes are satisfied by the current state.
            #If they are, add the resulting state to the queue (after consuming the recipe on the current state)
            
            for key in recipes:
                if (preconditions_satisfied(currentState, recipes[key])):
                
                    thisCost = cost
                    thisPath = path.copy()
                
                    #update the path taken and the cost to that path
                    thisPath.append(key)
                    thisCost += recipes[key].cost
                
                    #consume the state
                    newState = apply_effects(currentState, recipes[key])
                
                    #if the new state satisfies the goal, we are done!
                    if (newState >= goal):
                        return (visited, thisCost, thisPath)
                    
                    if (see_state(newState, all_combinations, seen_combinations)): #it's a unique state
                        #add it to the queue
                        open_list.append((visited, newState))
                        best_costs[newState] = thisCost
                        best_from[newState] = thisPath
    
    return visited, -1, None

In [16]:
#a depth-first search version of IW, uses a list like a stack

def width_search_DFS(initial: State, goal: State, WMax: int) -> Tuple[int, Optional[List[str]]]:
    start_time = time.time()
    all_propositions = recipe_propositions | state_propositions(initial) | state_propositions(goal)
    all_combinations: List[FrozenSet[Proposition]] = []
        
    visited = 0
    
    # Increase W up to WMax
    for W in range(1, WMax + 1):
        
        # Calculate all combinations of propositions at size W and add to all_combination
        all_combinations += [frozenset(props) for props in itertools.combinations(all_propositions, W)]
        print("W=",W,"Combination count=",len(all_combinations))
        
        # Track, for each combination (by index), whether we have seen this combination before (0 for no, >0 for yes)
        seen_combinations: Set[FrozenSet[Proposition]] = set()
            
        # Initialize seen_combinations
        #see_state(initial, all_combinations, seen_combinations)
        
        open_list: List[Tuple[int, State]] = [(0, initial)]
        best_costs: Dict[State, int] = {initial: 0}
        best_from: Dict[State, List[str]] = {initial: []}
            
        #TODO This should look like your graph search, except...
        #Call see_state on newly expanded states to update seen_combinations
        #and use its return value to decide whether to add this state to the 
        #open list (is that the only thing that determines whether it should go on the open list?)
            
        while open_list:
        
            #get the current state
            currentState = open_list.pop()[1] #always get the most recently added element in the list, like a stack
            
            cost = best_costs[currentState]
            path = best_from[currentState]
            visited += 1
        
            #check if any recipes are satisfied by the current state.
            #If they are, add the resulting state to the queue (after consuming the recipe on the current state)
            
            for key in recipes:
                if (preconditions_satisfied(currentState, recipes[key])):
                
                    thisCost = cost
                    thisPath = path.copy()
                
                    #update the path taken and the cost to that path
                    thisPath.append(key)
                    thisCost += recipes[key].cost
                
                    #consume the state
                    newState = apply_effects(currentState, recipes[key])
                
                    #if the new state satisfies the goal, we are done!
                    if (newState >= goal):
                        return (visited, thisCost, thisPath)
                    
                    if (see_state(newState, all_combinations, seen_combinations)): #it's a unique state
                        #add it to the queue
                        open_list.append((visited, newState))
                        best_costs[newState] = thisCost
                        best_from[newState] = thisPath
    
    return visited, -1, None

In [17]:
#breath-first search examples
print(width_search_BFS(State.from_dict({}),State.from_dict({'bench':1}),2))
print()
print(width_search_BFS(State.from_dict({'wood':1}),State.from_dict({'wooden_pickaxe':1}),3))
print()
print(width_search_BFS(State.from_dict({'wood':1}),State.from_dict({'iron_pickaxe':1}),4))
print()
print(width_search_BFS(State.from_dict({}),State.from_dict({'rail':1}),4))
print()
print(width_search_BFS(State.from_dict({}),State.from_dict({'cart':1}),4))

print("\n\n\n")

#depth-first search examples
print(width_search_DFS(State.from_dict({}),State.from_dict({'bench':1}),2))
print()
print(width_search_DFS(State.from_dict({'wood':1}),State.from_dict({'wooden_pickaxe':1}),3))
print()
print(width_search_DFS(State.from_dict({'wood':1}),State.from_dict({'iron_pickaxe':1}),4))
print()
print(width_search_DFS(State.from_dict({}),State.from_dict({'rail':1}),4))
print()
print(width_search_DFS(State.from_dict({}),State.from_dict({'cart':1}),4))
print()

W= 1 Combination count= 34
(3, 6, ['punch for wood', 'craft plank', 'craft bench'])

W= 1 Combination count= 34
W= 2 Combination count= 595
W= 3 Combination count= 6579
(31, 14, ['craft plank', 'craft stick', 'punch for wood', 'craft plank', 'punch for wood', 'craft bench', 'craft plank', 'craft wooden_pickaxe at bench'])

W= 1 Combination count= 34
W= 2 Combination count= 595
W= 3 Combination count= 6579
(606, 85, ['craft plank', 'craft stick', 'punch for wood', 'craft plank', 'punch for wood', 'craft bench', 'craft plank', 'craft wooden_pickaxe at bench', 'wooden_pickaxe for coal', 'wooden_pickaxe for cobble', 'wooden_pickaxe for cobble', 'wooden_pickaxe for cobble', 'craft stone_pickaxe at bench', 'stone_pickaxe for ore', 'stone_pickaxe for cobble', 'stone_pickaxe for cobble', 'stone_pickaxe for cobble', 'stone_pickaxe for cobble', 'stone_pickaxe for cobble', 'stone_pickaxe for cobble', 'stone_pickaxe for cobble', 'stone_pickaxe for cobble', 'craft furnace at bench', 'smelt ore in f

In [240]:
#a cell for calculating heuristic cost

#The goal state is boiled down to the raw ingredients needed to reach the goal state.
#i.e. a goal state of a bench can be boiled down to 4 planks, and then 4 planks to 1 wood.

#states closer to the ingredients of the goal state have a lower heuristic
#i.e. a state with 1 wood is better than an empty state, and a state with 4 planks is better than a state with 1 wood

#the closer a state is to the goal state, the better the heuristic!

def get_heuristic_h(currentState: State, goalState: State) -> int:

    #find all the recipes for the goal state
    goalStateRecipes = []
    plank = 0
    stick = 0

    for i in range(len(items_by_index)):
        if (goalState.items[i] > 0): #the state contains at least one of this item
            for key in recipes: #now search for the corresponding recipe for that item
                for r in range(len(items_by_index)):
                    if (recipes[key].produces.items[r] > 0):
                        
                        if (key == 'wooden_axe for wood' or
                            key == 'stone_axe for wood' or
                            key == 'iron_axe for wood' or
                            key == 'stone_pickaxe for cobble' or
                            key == 'iron_pickaxe for cobble' or
                            key == 'iron_pickaxe for ore'):
                            continue
                        
                        if(items_by_index[r] == items_by_index[i]): #the recipe for this item has been found!
                            for q in range(goalState.items[i]):
                                
                                if (key == 'craft plank'): #1 wood makes 4 planks
                                    if (plank % 4 == 0):
                                        goalStateRecipes.append(key)
                                    plank += 1
                                        
                                elif (key == 'craft stick'): #2 planks makes 4 sticks
                                    if (stick % 4 == 0):
                                        goalStateRecipes.append(key)
                                    stick += 1
                                    
                                else:
                                    goalStateRecipes.append(key)
                      
    #now we have all the recipes for the goal state
    #we need to recursively boil these recipes down to their component items,
    #until there are no more recipes that make up the resulting component items
        
    #if a recipe requires something, such as a bench, furnace, or tool, many recipes may need this,
    #so do not keep duplicates of required items, since only 1 is required for all the recipes that need this
    
    print(goalStateRecipes)
    
    requiredItems = [] #the required items will be added to this list if they are unique
    consumedItems = State.from_dict({}) #we will store the consumed items in this state by adding other states to it
    
    plank = 0
    stick = 0
        
    while goalStateRecipes: #loop until goalStateRecipes is empty
        
        #get what items are required and consumed from this recipe
        thisKey = goalStateRecipes.pop()
        consumedState = recipes[thisKey].consumes
        requiredState = recipes[thisKey].requires
        producedState = recipes[thisKey].produces
        
        #print("Consumes:", consumedState)
        #print("Requires:", requiredState)
        #print("Produces:", producedState)
        
        consumedItems += consumedState #add the consumed items for this recipe
        
        #add all required items for the recipes, if the items are unique
        for i in range(len(items_by_index)):
            if (requiredState.items[i] > 0 and (items_by_index[i] not in requiredItems)): #it's required AND unique
                requiredItems.append(items_by_index[i])
            
        #now add all the recipes for what is consumed back into goalStateRecipes
        for i in range(len(items_by_index)):
            if (consumedState.items[i] > 0):
                for key in recipes:
                    for r in range(len(items_by_index)):
                        if (recipes[key].produces.items[r] > 0):
                            
                            if (key == 'wooden_axe for wood' or
                                key == 'stone_axe for wood' or
                                key == 'iron_axe for wood' or
                                key == 'stone_pickaxe for cobble' or
                                key == 'iron_pickaxe for cobble' or
                                key == 'iron_pickaxe for ore'):
                                continue
                            
                            if(items_by_index[r] == items_by_index[i]):
                                for q in range(consumedState.items[i]):
                                    if (key == 'craft plank'): #1 wood makes 4 planks
                                        if (plank % 4 == 0):
                                            goalStateRecipes.append(key)
                                        plank += 1
                                            
                                    elif (key == 'craft stick'): #2 planks makes 4 sticks
                                        if (stick % 4 == 0):
                                            goalStateRecipes.append(key)
                                        stick += 1
                                        
                                    else:
                                        goalStateRecipes.append(key)
    
    return consumedItems

In [244]:
thisState = State.from_dict({})
goalState = State.from_dict({'stick':4})

print(get_heuristic_h(thisState, goalState))

['craft stick']
bench:0, cart:0, coal:0, cobble:0, furnace:0, ingot:0, iron_axe:0, iron_pickaxe:0, ore:0, plank:2, rail:0, stick:0, stone_axe:0, stone_pickaxe:0, wood:1, wooden_axe:0, wooden_pickaxe:0


In [20]:
#an alternate version of calculating the heuristic cost

#the following items lower the heuristic cost if the State has them:
#bench, furnace, iron_axe, iron_pickaxe, stone_axe, stone_pickaxe, wooden_axe, wooden_pickaxe

#tools should lower the heuristic cost of a state since the state now has more recipes/neighbors that it can visit,
#widening the frontier space and potentially leading it to the goal state

def get_heuristic(myState: State) -> int:
    
    num = 53
    
    if (myState.items[0] > 0): #bench
        num -= 15
        
    if (myState.items[4] > 0): #furnace
        num -= 10
        
    if (myState.items[16] > 0): #wooden_pickaxe
        num -= 5
        
    if (myState.items[13] > 0): #stone_pickaxe
        num -= 8
        
    if (myState.items[7] > 0): #iron_pickaxe
        num -= 8
    
    if (myState.items[15] > 0): #wooden_axe
        num -= 3
        
    if (myState.items[12] > 0): #stone_axe
        num -= 4
        
    #note: the iron axe was ommitted from this heuristic method because the iron axe can chop wood in the same time
         # a stone axe can chop wood, thus making it obsolete        
    
    return num #best heuristic cost is 0 if all the above items are in the state

In [21]:
#A* extra credit
#If an answer is found, it is saved and the algorithm proceeds to the next width size until all widths have been searched.
#This is because later width sizes may have more optimal answers, so all width sizes are run
#Once all width sizes have been searched, the algorithm returns the most optimal answer,
#or the answer with the least time to reach the goal state

import math

def A_star(initial: State, goal: State, WMax: int) -> Tuple[int, Optional[List[str]]]:
    start_time = time.time()
    all_propositions = recipe_propositions | state_propositions(initial) | state_propositions(goal)
    all_combinations: List[FrozenSet[Proposition]] = []
        
    visited = 0 #keep track of the number of states visited (kept between widths)
    answerStates = [] #a list to keep track of all answers found
    
    # Increase W up to WMax
    for W in range(1, WMax + 1):
        
        # Calculate all combinations of propositions at size W and add to all_combination
        all_combinations += [frozenset(props) for props in itertools.combinations(all_propositions, W)]
        print("W=",W,"Combination count=",len(all_combinations))
        
        # Track, for each combination (by index), whether we have seen this combination before (0 for no, >0 for yes)
        seen_combinations: Set[FrozenSet[Proposition]] = set()
        best_costs: Dict[State, int] = {initial: 0}
        best_from: Dict[State, List[str]] = {initial: []}
       
        priorityQueue = PriorityQueue()
        priorityQueue.put(initial, get_heuristic(initial)) #add the initial position to the queue
        
        #now check all nodes until the location has been found or the queue is empty
        while not priorityQueue.empty():
                
            #get the current state
            currentState = priorityQueue.get()[1]
            
            cost = best_costs[currentState] #retrieve the current cost and path
            path = best_from[currentState]
            visited += 1 #increment number of states visited
        
            #check if any recipes are "neighbors" of the current state
            #If they are, add the neighbor state to the queue (after consuming the recipe on the current state)
            
            for key in recipes:
                if (preconditions_satisfied(currentState, recipes[key])):
                
                    thisCost = cost #copy state variables so the original data is not changed
                    thisPath = path.copy()
                
                    #update the path taken and the cost to that path
                    thisPath.append(key)
                    thisCost += recipes[key].cost
                
                    #consume the state
                    newState = apply_effects(currentState, recipes[key])
                
                    #if the new state satisfies the goal, we are done!
                    if (newState >= goal):
                        answerStates.append((visited, thisCost, thisPath))
                        while not priorityQueue.empty():
                            priorityQueue.get()
                            
                        break
                    
                    if (see_state(newState, all_combinations, seen_combinations)): #it's a unique state
                        
                        try:
                            if (thisCost < best_costs[newState]):
                                #add it to the priority queue and update the best cost
                                priorityQueue.put(newState, thisCost + get_heuristic(newState))
                                best_costs[newState] = thisCost
                                best_from[newState] = thisPath
                                
                        except KeyError: #if this exception is raised, the distance is infinity, because the state has not yet
                                            #been added to the dictionary of best costs
                            
                            priorityQueue.put(newState, thisCost + get_heuristic(newState))
                            best_costs[newState] = thisCost
                            best_from[newState] = thisPath
    
    if not answerStates: #no answer state was found
        return visited, -1, None
    
    #return the lowest-cost state
    min = math.inf
    
    for i in answerStates:
        if (i[1] < min):
            min = i[1]
            
    for i in answerStates:
        if (i[1] == min):
            return i

In [28]:
print(A_star(State.from_dict({}), State.from_dict({'iron_pickaxe':1}), 3))
print()
print(A_star(State.from_dict({}), State.from_dict({'wood':50}), 2))

W= 1 Combination count= 34
W= 2 Combination count= 595
W= 3 Combination count= 6579
(743, 89, ['punch for wood', 'craft plank', 'craft bench', 'punch for wood', 'craft plank', 'craft stick', 'punch for wood', 'craft plank', 'craft wooden_pickaxe at bench', 'wooden_pickaxe for cobble', 'wooden_pickaxe for cobble', 'wooden_pickaxe for cobble', 'craft stone_pickaxe at bench', 'stone_pickaxe for cobble', 'stone_pickaxe for cobble', 'stone_pickaxe for cobble', 'stone_pickaxe for cobble', 'stone_pickaxe for cobble', 'stone_pickaxe for cobble', 'stone_pickaxe for cobble', 'stone_pickaxe for cobble', 'craft furnace at bench', 'wooden_pickaxe for coal', 'stone_pickaxe for ore', 'smelt ore in furnace', 'wooden_pickaxe for coal', 'stone_pickaxe for ore', 'smelt ore in furnace', 'wooden_pickaxe for coal', 'stone_pickaxe for ore', 'smelt ore in furnace', 'craft stick', 'craft iron_pickaxe at bench'])

W= 1 Combination count= 83
W= 2 Combination count= 3486
(676, 81, ['punch for wood', 'craft plank'